# Interactive calculator!

### To use: Run packlages and Code first, then tweak the numbers in the last cell 

#### Packages

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets

In [143]:
def chart_costs_delux(n,price=500000,
                      interestRate= 7.21,
                      downPayment=200000,
                     loanTerm= 30,
                     insurancePcnt= .04,
                     hoa=360,
                    pctAnnualGrowth=.05,
                    rentMonthly=3500,
                    millage=18.42,
                     maintenance=7000,
                     year_compare=5,
                      incomeTax=.24):
    # Loan Amount
    loanAmt= price-downPayment
    
    interestDeductionRate=1-incomeTax
    # Interest Total Cost
    interestCost=(interestRate/100)*loanAmt
    # Monthly mortgage
    #mortgageMonthly=(loanAmt+interestCost)/(loanTerm*nAnnualPayments)
    #mortgageInterestYearly=interestCost/(loanTerm*nAnnualPayments)

    # propTax (percent)
    taxYearly=price*(millage/1000)
    taxMonthly=taxYearly/12

    #Monthly insurance
    insuranceYearly=price*insurancePcnt
    # HOA (monthly)
    hoaYearly=hoa*12


    def calculate_amortization_schedule(loan_amount,  interestRate, loan_term_years):
        monthly_rate =  interestRate / 100 / 12
        total_payments = loan_term_years * 12
    
        # Calculate fixed monthly mortgage payment
        monthly_payment = loan_amount * monthly_rate / (1 - (1 + monthly_rate) ** -total_payments)
    
        schedule = []
        balance = loan_amount
    
        for month in range(1, total_payments + 1):
            interest = balance * monthly_rate
            principal = monthly_payment - interest
            balance -= principal
    
            schedule.append({
                "Month": month,
                "Payment": round(monthly_payment, 2),
                "Principal": round(principal, 2),
                "Interest": round(interest, 2),
                "Balance": round(balance if balance > 0 else 0, 2)
            })
    
        return schedule
    
    def get_values_from_key(list_of_dictionaries, key):
        """
        Extracts all values associated with a specific key from a list of dictionaries.
    
        Args:
            list_of_dictionaries: A list of dictionaries.
            key: The key to extract values for.
    
        Returns:
            A list containing all values associated with the given key.
        """
        return [dictionary[key] for dictionary in list_of_dictionaries if key in dictionary]
        
    def average_chunks(X, chunk_size=12):
        """
        Averages every `chunk_size` consecutive items in list X.
    
        :param X: List of numbers (length must be divisible by chunk_size)
        :param chunk_size: Number of items to average together
        :return: List of averaged values
        """
        if len(X) % chunk_size != 0:
            raise ValueError("List length must be divisible by chunk size.")
    
        return [
            sum(X[i:i + chunk_size]) / chunk_size
            for i in range(0, len(X), chunk_size)
        ]
    
    def Sum_years(sched,Var):
        Val=get_values_from_key(sched,Var)
        return average_chunks(Val, chunk_size=12)

    def calc_investment_growth(p, pctAnnualGrowth, y):
        """
        Calculates the future value of an investment using compound interest.
    
        :param price: Initial investment amount (P)
        :param pctAnnualGrowth: Annual interest rate (in percent, e.g. 5 for 5%)
        :param y: Number of years the money is invested (t)
        :param compounds_per_year: Number of times interest is compounded per year (n)
        :return: Future value of the investment (A)
        """
        growth = (p * (1 + pctAnnualGrowth) **  y)-p
        #growth = p * (1 + pctAnnualGrowth / compounds_per_year) ** (compounds_per_year * y)
        return growth
    
    def create_growthChart(n):
        yearList=range(n)
        growthChart = []
        for x in yearList:
            growthChart.append(round(calc_investment_growth(price, pctAnnualGrowth, x)))
        return growthChart
    
    
    schedule=calculate_amortization_schedule(loanAmt, interestRate, loanTerm)
    
    mortgageYearly=Sum_years(schedule,"Payment")
    
    interestYearly=Sum_years(schedule,"Interest")
    interestDeduction=[x*interestDeductionRate for x in interestYearly ]
    principalYearly=Sum_years(schedule,"Principal")
    taxAdjustedYearly=[x + y for x, y in zip(principalYearly, interestDeduction)]

    
    def calc_cost(y,principal,iYearly,tYearly,hYearly,maint):
        cost=sum(principal[:y])+((iYearly+tYearly+hYearly+maint)*y)
        return cost
    
    def create_costChart(n,principal,iYearly,tYearly,hYearly,maint):
        yearList=range(n)
        costChart = []
        for x in yearList:
            costChart.append(round(calc_cost(x,principal,iYearly,tYearly,hYearly,maint)))
        return costChart
    
    def calc_rent(y):
        rent=y*12*rentMonthly
        return rent
    
    def create_rentChart(n):
        yearList=range(n)
        rentChart=[]
        for x in yearList:
            rentChart.append(round(calc_rent(x)))
        return rentChart
    
    y1 = np.array(create_costChart(n,taxAdjustedYearly,insuranceYearly,taxYearly,hoaYearly,maintenance))
    y2 = np.array(create_growthChart(n))
    y3 = np.array(create_rentChart(n))
    plt.plot(y1,label="costs")
    plt.plot(y2,label="gains")
    plt.plot(y3,label="rent spent")
    plt.legend(loc="upper left")
    plt.ticklabel_format(axis='both', style='plain',)
    monthly_cost=round(np.array(create_costChart(n,Sum_years(schedule,"Payment"),insuranceYearly,taxYearly,hoaYearly,maintenance))[1]/12,2)

    earnings=round(calc_investment_growth(price, pctAnnualGrowth, year_compare))-round(calc_cost(year_compare,taxAdjustedYearly,insuranceYearly,taxYearly,hoaYearly,maintenance))

    plt.plot([year_compare,year_compare],
             [round(calc_investment_growth(price, pctAnnualGrowth, year_compare)),
              round(calc_cost(year_compare,taxAdjustedYearly,insuranceYearly,taxYearly,hoaYearly,maintenance))],
             c="y")
    plt.text(year_compare+5,round(calc_cost(year_compare,taxAdjustedYearly,insuranceYearly,taxYearly,hoaYearly,maintenance)), 'earnings:' + '\n' + str(earnings), size=10, rotation=0,
                      ha="center", va="center",
         bbox=dict(boxstyle="round",
                   ec=(1., 0.5, 0.5),
                   fc=(1., 0.8, 0.8),
                   )
         )

    plt.text(year_compare+5,0, 'rent:' + '\n' + str(-1*round(calc_rent(year_compare))), size=10, rotation=0,
                      ha="center", va="center",
         bbox=dict(boxstyle="round",
                   ec=(1., 0.5, 0.5),
                   fc=(1., 0.8, 0.8),
                   )
         )
    plt.text(2, 450000, 'monthly costs:' + '\n' + str(monthly_cost), size=10, rotation=0,
                      ha="center", va="center",
         bbox=dict(boxstyle="round",
                   ec=(1., 0.5, 0.5),
                   fc=(1., 0.8, 0.8),
                   )
         )
    plt.text(2, 300000, 'monthly mortgage:' + '\n' + str(get_values_from_key(schedule,"Payment")[1]), size=10, rotation=0,
         ha="center", va="center",
         bbox=dict(boxstyle="round",
                   ec=(1., 0.5, 0.5),
                   fc=(1., 0.8, 0.8),
                   )
         )
    
    plt.show()

### Change these numbers to fit you! 

#### (btw Millage is property tax per thousand dollars of property value, federal income tax [incomeTax] set at twenty four %, hoa is Home Owners Association fees per month)

In [144]:
 ipywidgets.interact(chart_costs_delux,n=(1,30,1),price=(0,2000000,10000),
                      interestRate= (3,15,.1),# 7.21,
                      downPayment=(0,1000000,10000),#200000,
                     loanTerm= [5,30],
                     insurancePcnt= [.006,.025],
                     incomeTax=[.24],
                     hoa=(0,1500,20),
                    pctAnnualGrowth=(-.20,.11,.01),
                    rentMonthly=3500,
                    millage=(15,20,.01),
                    maintenance=(1000,10000,1000),
                    year_compare=(1,30,1)
)

interactive(children=(IntSlider(value=15, description='n', max=30, min=1), IntSlider(value=500000, description…

<function __main__.chart_costs_delux(n, price=500000, interestRate=7.21, downPayment=200000, loanTerm=30, insurancePcnt=0.04, hoa=360, pctAnnualGrowth=0.05, rentMonthly=3500, millage=18.42, maintenance=7000, year_compare=5, incomeTax=0.24)>